In [ ]:
from agents.basic_agents import BasicAgent
from utilities import print_long_text

from config.project_paths import synthetic_data_dir

this notebook holds a function to run rag, evaluation and requerying 

code should be folded into an agent

In [ ]:
agent = BasicAgent()

In [ ]:
import logging
import re

logging.basicConfig(
    format='%(name)s-%(levelname)s|%(lineno)d:  %(message)s', level=logging.INFO)
log = logging.getLogger(__name__)


def reretrieve__toc(question):
    documents = agent.retriever._get_relevant_documents(question)

    answer = agent.answer_question(
        question, documents)

    evaluation = agent.evaluate_answer(question, answer)
    
    
    match = re.search(r"Score \(1-5\): (\d+)", evaluation.content)

    if match:
        score = match.group(1)
        print(f"Extracted score: {score}")

        try:
            if int(score.strip()) < 4:
                new_answer = agent.regenerate_new_answer_from_evaluation(
                    question, answer, evaluation)
                print('\n\n regenerated answer:')
                print_long_text(new_answer.content)
        except Exception as e:
            print(e)

    else:
        print("Score not found in the string.")

    print('\n\ndocs:')

    for i in documents:
        print(i.metadata)
        print_long_text(i.page_content)

In [ ]:
data_dir = synthetic_data_dir / 'search_language_qna'

qna_files = list(data_dir.rglob('*.csv'))
len(qna_files)

In [ ]:
from synthetic_data_generation.generate_syn_data import csv_to_list

In [ ]:
import random

random_qna_sample = random.sample(qna_files, 15)

qnas = []
for i in random_qna_sample:
    qnas.extend(random.sample(csv_to_list(i), 4))


In [ ]:
qnas

In [ ]:
from agents.rag_agents import RagAgent


rag_agent = RagAgent()
responses = []

for question, answer in qnas:
    print('question:')
    print(question)
    
    print('\n\nground truth:')
    print(answer)

    response = rag_agent.answer_with_rag(question)
    responses.append(response)
    
    print(f'\n\ngenerated answer:{response.answer}')
    print(f'\n\nevaluation score:{response.evaluation_score}')

    
    print('\n\n')

In [ ]:
from dataclasses import asdict

t = ''
for response, qna in zip(responses, qnas):
    for k,v in asdict(response).items():
        t += k.capitalize() + '\n'
        t += '-'*40 + '\n'
        if k.lower().strip() == 'documents':
            t += '\n\n'.join([i.page_content for i in v]) + '\n\n'
        elif k.lower().strip() == 'evaluation': 
            t += v.content + '\n\n'
        elif k.lower().strip() == 'answer':
            t += v.content + '\n\n'
        else:
            t += str(v) + '\n\n'
        
    t += 'ground truth answer'.capitalize() + '\n'
    t += qna[1] + '\n\n'
    
    t += '='*100
    t += '\n\n'

In [ ]:
from config.project_paths import project_root

o = project_root/'synthetic_data_generation'/'trial_search_engine_lang.txt'
o.write_text(t)

In [ ]:
response = rag_agent.answer_with_rag('implement eks observability')
print_long_text(response.answer.content)

In [ ]:
response.evaluation_score